## Clause Risk Categorization

Putting each of the TCLP clauses into risk categories using an LLM 

In [1]:
import pandas as pd
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
import utils

In [2]:
risk_taxonomy = pd.read_excel('../data/risk_taxonomy.xlsx')

In [6]:
clause_folder = "../data/cleaned_content"
clause_html = '../data/clause_boxes'
model_path = "../models/CC_BERT/CC_model_detect"

In [7]:
tokenizer, d_model, c_model, names, docs, final_df = utils.getting_started(model_path, clause_folder, clause_html)

/Users/georgia/Documents/coding/climate_risk_id/tclp/utils.py:557: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(content, "html.parser")


In [8]:
# make a df of names and docs
df = pd.DataFrame({'name': names, 'clause': docs})

In [9]:
risk_taxonomy

,Label,Description
0,Physical-flooding,Clause that helps reduce exposure to flooding ...
1,Physical-wildfire,Clause that helps mitigate exposure to wildfir...
2,Physical-heat,Clause that helps reduce exposure to overheati...
3,Physical-subsidence,Clause that helps reduce exposure to ground in...
4,Physical-sea-level,Clause that helps reduce exposure to coastal e...
5,Physical-water-scarcity,Clause that helps reduce exposure to water str...
6,Physical-extreme-weather,"Clause that helps reduce exposure to storm, wi..."
7,Physical-infrastructure,Clause that helps reduce exposure to infrastru...
8,Physical-general,Clause that helps manage general exposure to p...
9,Transition-mees,Clause that helps reduce exposure to MEES-rela...


In [15]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"), 
    base_url = "https://openrouter.ai/api/v1"
)

In [16]:
messages = "You are a helpful assistant whose job it is to identify the risk type given a provided clause. These clauses WILL NOT contain the risk themselves. Rather, they are designed to help legal users to mitigate risk. So you are meant to identify the risk categorizations that the given clause might help protect against. Feel free to pick more than one risk that you think the clause could be relevant for."

In [17]:
clause_1 = df.iloc[0]['name'] + df.iloc[0]['clause'] 

In [18]:
result = utils.classify_clause(clause_1, risk_taxonomy, messages, client)

In [29]:
print(result)

{
  "labels": ["Transition-retrofit", "Transition-standards", "Transition-disclosure"],
  "justification": "The clause provides a detailed guide and checklist for accessing Sustainability-Linked Loans (SLLs), which helps companies align their financial strategies with net zero transition goals. It includes setting and reporting on Sustainability Performance Targets (SPTs), which are crucial for transition-related risks. The clause also emphasizes the importance of disclosing sustainability standards and certifications, which helps mitigate transition-disclosure risks. Additionally, it encourages the use of third-party verification and setting ambitious targets, which are key aspects of transition-retrofit and transition-standards."
}


____

## Creating database and applying this to all clauses

In [30]:
# Get the list of all possible risk labels (from your taxonomy)
risk_labels = list(risk_taxonomy['Label'].str.strip())

In [31]:
results_df = pd.DataFrame(columns=['name'] + risk_labels + ['justification'])

In [33]:
for i, row in df.iterrows():
    clause_text = row['name'] + row['clause']
    result = utils.classify_clause(clause_text, risk_taxonomy, messages, client)
    
    print(f"Processing clause {i+1}/{len(df)}: {row['name']}")
    
    # Format the result
    formatted_row = utils.format_classification_result(row['name'], result, risk_labels)
    
    # Append to the DataFrame
    results_df = pd.concat([results_df, pd.DataFrame([formatted_row])], ignore_index=True)

Processing clause 1/122: A Beginner’s Guide and Checklist for Accessing Sustainability-Linked Loans (SLLs)
Processing clause 2/122: Allocating Scope 1, 2 and 3 Emissions for Leased Assets
Processing clause 3/122: Auditing Water Usage in Supply Chains
Processing clause 4/122: Avoiding Excessive Paperwork in Dispute Resolution
```json
{
  "labels": ["Legal-disclosure", "Legal-general"],
  "justification": "This clause helps mitigate legal exposure related to disclosure and general climate risks. It ensures that parties handle disputes in a manner that reduces environmental impact and includes steps to offset emissions. This approach can help reduce the risk of legal claims arising from the failure to disclose or manage climate risks properly. The clause also provides a framework for addressing and mitigating the environmental impact of disputes, which can be seen as a broader legal risk management strategy."
}
```
Processing clause 5/122: Benchmarking of Project Greenhouse Gas Emissions


In [34]:
results_df

,name,Physical-flooding,Physical-wildfire,Physical-heat,Physical-subsidence,Physical-sea-level,Physical-water-scarcity,Physical-extreme-weather,Physical-infrastructure,Physical-general,...,Legal-tort-liability,Legal-access,Legal-contract,Legal-penalties,Legal-negligence,Legal-insurance,Legal-disclosure,Legal-breach,Legal-general,justification
0,A Beginner’s Guide and Checklist for Accessing...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The clause provides a detailed guide and check...
1,"Allocating Scope 1, 2 and 3 Emissions for Leas...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,This clause helps mitigate transition-related ...
2,Auditing Water Usage in Supply Chains,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,The clause focuses on auditing and reducing wa...
3,Avoiding Excessive Paperwork in Dispute Resolu...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Invalid JSON response
4,Benchmarking of Project Greenhouse Gas Emissions,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,This clause helps mitigate risks related to fu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Target Product Carbon Footprint (Schedule for ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The clause focuses on setting and reducing the...
118,Template Board Paper for Significant Contracts...,0,0,0,0,0,0,0,0,1,...,0,0,1,0,1,0,1,0,0,This clause helps mitigate a broad range of ri...
119,The Net Zero Standard for Suppliers,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,This clause helps mitigate exposure to future ...
120,The ‘Green Supplier’ Contract – A Standardised...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Invalid JSON response


In [ ]:
#add a column for having any label 
results_df['any_label'] = results_df[risk_labels].any(axis=1)

In [38]:
results_df.any_label.value_counts()

any_label
True    122
Name: count, dtype: int64

In [ ]:
#save this CSV 
results_df.to_csv('risk_classification_results.csv', index=False)